In [3]:
import sys
sys.path
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.applications.vgg16 import VGG16
from keras.utils import np_utils
import keras
keras.backend.set_image_data_format('channels_first')

Using TensorFlow backend.


In [4]:
import keras.backend as K
import tensorflow as tf

# Custom metrics

# f1 score
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


In [5]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [6]:
def vgg16_model(img_rows, img_cols, channel, num_classes):
    """VGG 16 Model for Keras
    Model Schema is based on 
    https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
    ImageNet Pretrained Weights 
    https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view?usp=sharing
    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of categories for our classification task
    """
    model = Sequential()
    
    #1
    model.add(ZeroPadding2D((1, 1), input_shape=(channel, img_rows, img_cols)))
    #first convolutional layer, the network has to learn 64 filters with size 3x3 along the input depth (3) 
    #each one of the 64 filters has bias, so the total number of parameters is: 64*3*3*3 + 64 = 1792
    model.add(Convolution2D(64, 3, 3, activation='relu', name = 'block1_conv1'))
    

    model.add(ZeroPadding2D((1, 1)))
    #second convolutional layer, the network has to learn 64 filters with size 3x3 along the input depth (3) 
    #each one of the 64 filters has bias, so the total number of parameters is: 64*3*3*3 + 64 = 1792
    model.add(Convolution2D(64, 3, 3, activation='relu', name ='block1_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    #2
    model.add(ZeroPadding2D((1, 1)))
    #third convolutional layer, the network has to learn 128 filters with size 3x3 along the input depth (3) 
    #each one of the 128 filters has bias, so the total number of parameters is: 128*3*3*3 + 128 = 3584
    model.add(Convolution2D(128, 3, 3, activation='relu', name ='block2_conv1'))
    
    model.add(ZeroPadding2D((1, 1)))
    #fourth convolutional layer, the network has to learn 128 filters with size 3x3 along the input depth (3) 
    #each one of the 128 filters has bias, so the total number of parameters is: 128*3*3*3 + 128 = 3584
    model.add(Convolution2D(128, 3, 3, activation='relu', name ='block2_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="th"))

    #3
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name ='block3_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name ='block3_conv2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name ='block3_conv3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="th"))

    #6
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name ='block4_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name ='block4_conv2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name ='block4_conv3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="th"))

    #7
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name ='block5_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name ='block5_conv2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name ='block5_conv3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="th"))

    # Add Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    # Loads ImageNet pre-trained data
    model.load_weights('vgg16_weights.h5')

    # Truncate and replace softmax layer for transfer learning
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    model.add(Dense(num_classes, activation='softmax'))

    # Uncomment below to set the first 10 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:10]:
        layer.trainable = False

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-6, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])#,f1])

    return model

In [7]:
import cv2
import numpy as np
import h5py
import matplotlib.pyplot as plt
from skimage.color import rgb2gray

from keras import backend as K
from keras.utils import np_utils

num_train_samples = 0
num_valid_samples = 0
num_classes = 2

def grayscale(img):
    # We need to normalize our values between 1 and -1
    R = np.max(np.abs(img[:,:,0]))
    G = np.max(np.abs(img[:,:,1]))
    B = np.max(np.abs(img[:,:,2]))
    
    BlackMax = 0.2125*R + 0.7154*G + 0.0721*B
    img[:,:,0] /= R
    img[:,:,1] /= G
    img[:,:,2] /= B
    
    return rgb2gray(img)*BlackMax

def normalise(img, color):
    mean_pixel = [103.939, 116.779, 123.68] # These values are obtained from the pretrained model
    img = img.astype(np.float32, copy=False)
    for c in range(3):
        img[:, :, c] = img[:, :, c] - mean_pixel[c]
    img = img.transpose((2,0,1))
    #img = np.expand_dims(img, axis=0)
    
    if(color):
        return img
    
    else:
        return grayscale(img)

def load_image_data(openClusters, globularClusters,channel):
    percentTrain = 0.8
    percentValidation = 0.1
    percentTest = 0.1
    
    # Toggle color /grayscale
    if(channel == 3):
        color = True
    if(channel == 1):
        color = False
    
    X_train = []
    Y_train =  []
    X_validation =  []
    Y_validation =  []
    X_test =  []
    Y_test =  []
    
    # Separate open clusters into train, validation, test
    for (i,img) in enumerate(openClusters):
        # Include open clusters in training set
        if(i < len(openClusters)*percentTrain):
            X_train.append(normalise(img,color))
            Y_train.append(0)
            
        # Include open clusters in validation set
        if(i >= len(openClusters)*percentTrain and i < len(openClusters)*(percentTrain+percentValidation)):
            X_validation.append(normalise(img,color))
            Y_validation.append(0)
        
        # Include open clusters in validation set
        if(i >= len(openClusters)*(percentTrain+percentValidation) and i <= len(openClusters)*(percentTrain+percentValidation + percentTest)):
            X_test.append(normalise(img,color))
            Y_test.append(0)
    
    # Separate globular clusters into train, validation, test
    for (i,img) in enumerate(globularClusters):
        # Include open clusters in training set
        if(i < len(globularClusters)*percentTrain):
            X_train.append(normalise(img,color))
            Y_train.append(1)
            
        # Include open clusters in validation set
        if(i >= len(globularClusters)*percentTrain and i < len(globularClusters)*(percentTrain+percentValidation)):
            X_validation.append(normalise(img,color))
            Y_validation.append(1)
        
        # Include open clusters in validation set
        if(i >= len(globularClusters)*(percentTrain+percentValidation) and i <= len(globularClusters)*(percentTrain+percentValidation + percentTest)):
            X_test.append(normalise(img,color))
            Y_test.append(1)
    
    X_train = np.asarray(X_train,dtype = 'uint8')
    X_validation = np.asarray(X_validation,dtype = 'uint8')
    X_test = np.asarray(X_test,dtype = 'uint8')
    Y_train = np.asarray(Y_train,dtype = 'int')
    Y_validation = np.asarray(Y_validation,dtype = 'int')
    Y_test = np.asarray(Y_test,dtype = 'int')
    
    Y_train = np_utils.to_categorical(Y_train,num_classes)
    Y_validation = np_utils.to_categorical(Y_validation,num_classes)
    Y_test = np_utils.to_categorical(Y_test,num_classes)
    
    return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [8]:
filename = 'OpC.h5'
openClustersH5 = h5py.File(filename, 'r')
nImages = 4000
openClusters = [openClustersH5.get("image"+str(i))[:] for i in range(0,nImages)]
openClustersH5.close()

filename = 'GlCl.h5'
globularClustersH5 = h5py.File(filename, 'r')
nImages = 4000
globularClusters = [globularClustersH5.get("image"+str(i))[:] for i in range(0,nImages)]
globularClustersH5.close()


In [9]:
# Model visualization
import matplotlib.pyplot as plt

# Plot model loss, accuracy and f1 in terms of epoch
def plotProgress(history):
    # list all data in history
    print(history.history.keys())
    
    
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:


if __name__ == '__main__':

    # Example to fine-tune on 3000 samples from Cifar10

    img_rows, img_cols = 224, 224 # Resolution of inputs
    channel = 3
    num_classes = 2
    batch_size = 16
    #10
    nb_epoch = 40

    # Load Cifar10 data. Please implement your own load_data() module for your own dataset
    X_train, Y_train, X_valid, Y_valid , X_test, Y_test = load_image_data(openClusters, globularClusters, channel = channel)

    # Load our model
    model = vgg16_model(img_rows, img_cols, channel, num_classes)
    model.summary()

    # Start Fine-tuning
    history = model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(X_valid, Y_valid),
              )

    # Make predictions
    predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

    #add grad_cam implementation here 
    # Cross-entropy loss score
    #score = log_loss(Y_valid, predictions_valid)

/home/jsgrafix26/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), name="block1_conv1", activation="relu")`
/home/jsgrafix26/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), name="block1_conv2", activation="relu")`
/home/jsgrafix26/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), name="block2_conv1", activation="relu")`
/home/jsgrafix26/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), name="block2_conv2", activation="relu")`
/home/jsgrafix26/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), strides=(2, 2), data_forma

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
__________

/home/jsgrafix26/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 6400 samples, validate on 800 samples
Epoch 1/40
6400/6400 [==============================] - 50s 8ms/step - loss: 0.6934 - acc: 0.4764 - val_loss: 0.6922 - val_acc: 0.5250
Epoch 2/40
6400/6400 [==============================] - 46s 7ms/step - loss: 0.6921 - acc: 0.5334 - val_loss: 0.6910 - val_acc: 0.5825
Epoch 3/40
6400/6400 [==============================] - 46s 7ms/step - loss: 0.6913 - acc: 0.5692 - val_loss: 0.6900 - val_acc: 0.6400
Epoch 4/40
6400/6400 [==============================] - 46s 7ms/step - loss: 0.6904 - acc: 0.6086 - val_loss: 0.6892 - val_acc: 0.6650
Epoch 5/40
6400/6400 [==============================] - 46s 7ms/step - loss: 0.6899 - acc: 0.6308 - val_loss: 0.6884 - val_acc: 0.6913
Epoch 6/40
6400/6400 [==============================] - 46s 7ms/step - loss: 0.6890 - acc: 0.6577 - val_loss: 0.6876 - val_acc: 0.7125
Epoch 7/40
6400/6400 [==============================] - 46s 7ms/step - loss: 0.6885 - acc: 0.6723 - val_loss: 0.6868 - val_acc: 0.7438
Epoch 8/

In [ ]:
def overlay(array1, array2, alpha=0.5):
    """Overlays `array1` onto `array2` with `alpha` blending.
    Args:
        array1: The first numpy array.
        array2: The second numpy array.
        alpha: The alpha value of `array1` as overlayed onto `array2`. This value needs to be between [0, 1],
            with 0 being `array2` only to 1 being `array1` only (Default value = 0.5).
    Returns:
        The `array1`, overlayed with `array2` using `alpha` blending.
    """
    if alpha < 0. or alpha > 1.:
        raise ValueError("`alpha` needs to be between [0, 1]")
    if array1.shape != array2.shape:
        raise ValueError('`array1` and `array2` must have the same shapes')

    return (array1 * alpha + array2 * (1. - alpha)).astype(array1.dtype)

In [ ]:
import matplotlib.cm as cm
from vis.visualization import visualize_cam
from matplotlib import pyplot as plt
import matplotlib

glIndices = [0,2]
opIndices = [21,24]

for modifier in [None]:#, 'guided', 'relu']:
    plt.figure(figsize = (20,90))
    f, ax = plt.subplots(1,5)
    #plt.suptitle("vanilla" if modifier is None else modifier)
    for i in range(5):
        img = openClusters[i+25]
        grads = visualize_cam(model, layer_idx=-1, filter_indices=0,
                                  seed_input=img, backprop_modifier=modifier)
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
#         print(np.shape(img))
#         print(np.shape(jet_heatmap))
#         print(i)
        #x = np.zeros(jet_heatmap)
        #result = [:,:,:,0]
        ax[i].imshow(overlay(jet_heatmap[:,:,:,0],img))

In [ ]:
img = openClusters[21]
grads = visualize_cam(model, layer_idx=-1, filter_indices=0,
                          seed_input=img, backprop_modifier=modifier)
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
o1im = img
o1 = overlay(jet_heatmap[:,:,:,0],img)

img = openClusters[24]
grads = visualize_cam(model, layer_idx=-1, filter_indices=0,
                          seed_input=img, backprop_modifier=modifier)
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
o2im = img
o2 = overlay(jet_heatmap[:,:,:,0],img)

img = globularClusters[0]
grads = visualize_cam(model, layer_idx=-1, filter_indices=0,
                          seed_input=img, backprop_modifier=modifier)
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
g1im = img
g1 = overlay(jet_heatmap[:,:,:,0],img)

img = globularClusters[2]
grads = visualize_cam(model, layer_idx=-1, filter_indices=0,
                          seed_input=img, backprop_modifier=modifier)
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
g2im = img
g2 = overlay(jet_heatmap[:,:,:,0],img)

In [ ]:
fig, ax = plt.subplots(2, 4)
fig.set_size_inches(22,12)

ax[0,0].imshow(o1im)
ax[0,0].set_title('Original open cluster', size = 22)
ax[0,0].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off

ax[0,1].imshow(o2im)
ax[0,1].set_title('Original open cluster', size = 22)
ax[0,1].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off

ax[0,2].imshow(g1im)
ax[0,2].set_title('Original globular cluster', size = 22)
ax[0,2].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off

ax[0,3].imshow(g2im)
ax[0,3].set_title('Original globular cluster', size = 22)
ax[0,3].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off


ax[1,0].imshow(o1)
ax[1,0].set_title('VGG16 heat map', size = 22)
ax[1,0].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off

ax[1,1].imshow(o2)
ax[1,1].set_title('VGG16 heat map', size = 22)
ax[1,1].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off

ax[1,2].imshow(g1)
ax[1,2].set_title('VGG16 heat map', size = 22)
ax[1,2].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off

ax[1,3].imshow(g2)
ax[1,3].set_title('VGG16 heat map', size = 22)
ax[1,3].tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left = False,
    right = False,
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft = False) # labels along the bottom edge are off


In [ ]:
# https://stackoverflow.com/questions/42763094/how-to-save-final-model-using-keras
# keras library import  for Saving and loading model and weights


from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_num.h5")